# Fourier transforms of simple 2D functions

So, in our application of optics, we will not be dealing with one dimensional plots but with two dimensional images instead. This means that our function $f$ in direct space is a function of **two** independent spatial variables, $f = f(x, y)$, and the FT of that will also have two independent spatial frequency variables, $F = F(s, r)$.

The two-dimensional Fourier transform does exactly the same like the one-dimensional FT, decompose a signal in its sinusoidal components except that we now have to think all across a 2D surface as opposed to a single, 1D line.

We have not touched yet any of the important basic theorems, but I though it is really important to get the visualizations for 2D transforms in, especially since this will set us up for jumping into Fourier Optics and from then on work with the optical application on our minds.

The analytical definition of the 2D FT is:

$$F(s, r) = \mathscr{F}\{f(x, y)\}(s, r) = \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} f(x, y) e^{-i 2 \pi (x s+yr)} dx dy$$

There are many functions $f(x,y)$ that have a well-defined analytical Fourier transform, but I will refrain from showing them here and focus only on the numerical implementation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D   # this is for surface plots
from matplotlib import cm
%matplotlib inline

Like for the 1D case, we will define some convenience functions that will perform a 2D Fourier Transform with the respective array shifts needed. And I define some utility functions.

In [ ]:
def ft2d(func):
    ft = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(func)))
    return ft

def ift2d(func):
    ift = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(func)))
    return ift

def ft1d_freq(x):
    """Calculate the (spatial) frequency array based on the spatial array x."""
    s = np.fft.fftshift(np.fft.fftfreq(x.size, d=x[-1]-x[-2]))
    return s

def zoom(im, x, y, bb):
    """Cut out a square box from image im centered on (x,y) with half-box size bb."""
    return(im[y-bb:y+bb,x-bb:x+bb])

def padcplx(c, pad=5):
    """Puts a Complex array in the centre of a zero-filled Complex array.
    pad defines the padding multiplier for the output array."""
    (nx, ny) = c.shape
    bignx = nx * pad + 1
    bigny = ny * pad + 1
    big_c = np.zeros((bignx, bigny),dtype=complex)
    
    dx = int((nx * (pad-1)) / 2 + 1)
    dy = int((ny * (pad-1)) / 2 + 1)
    
    big_c[dx:dx+nx,dy:dy+ny] = c
    return(big_c)

## Setting up the grids

We will first generate a grid for the independent variables $x$ and $y$. I chose the grid to have unity size and go from -0.5 to 0.5 so that I only ever have to define the sizes of my functions $f(x,y)$ with respect to the total grid.

In [ ]:
# We first need to generate the independent variables,
# this time as a 2D grid.
npix = 512

lin = np.linspace(-0.5, 0.5, npix)
xx, yy = np.meshgrid(lin, lin)

print("Shape of xx: {}".format(xx.shape))
print("Shape of yy: {}".format(yy.shape))
print("xx[:3,0]: {}".format(xx[:3,0]))
print("xx[-3:,0]: {}".format(xx[-3:,0]))

plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.imshow(xx)
plt.title('xx')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(yy)
plt.title('yy')
plt.colorbar()

We will also set up the spatial frequency arrays for the variables $s$ and $r$.  

Now, one thing that I don't want to get into in this notebook just yet is the subject of *padding* and *sampling*. I was able to get away with ignoring these things in the 1D cases because I just picked my different $f(x)$ in such a way that we were never undersampled and I never had to tweak the plots much in order to get nice displays. This gets more complicated in the 2D case and while I could still be smart about it in the same way, the plotting will need major adjustments, so I can just as well add the padding now to make my life easier.

Again, I will come back to this at a later point, the short version is that we need to shove our $f(x,y)$ images into bigger arrays that pad the function before performing a Fourier transform. This means that the data points we are interested in only take up a small fraction of the arrays we're dealing with, and I will need to either adjust the plot ranges or specifically cut out the data parts that we want to inspect.

In [ ]:
pad = 5   # factor by how much do we pad our images before performing a FT

npix_pad = npix*pad+1   # figure out the padded big array sizes after the FT
xsf = np.linspace(-0.5, 0.5, npix_pad)
gf = ft1d_freq(xsf)
ss, rr = np.meshgrid(gf, gf)   # create the spatial frequency grids

print("Shape of ss: {}".format(ss.shape))
print("Shape of rr: {}".format(rr.shape))

plt.figure(figsize=(10,10))
plt.subplot(1, 2, 1)
plt.imshow(xx)
plt.title('ss')
plt.subplot(1, 2, 2)
plt.imshow(yy)
plt.title('rr')

## A 2D rectangle - a rectangular aperture

### Numerical representation of the rectangle function

We can represent a rectangular aperture with a 2D function where the function value at each point on the grid is the amplitude of the aperture at that point. In the application of optics, simple shapes like squares, rectangles or circles are usually the functions used to describe pupil apertures, so the amplitude of a function like that is equivalent to the transmission of the aperture. In the simplest case, the aperture function will have non-zero values inside the aperture where it is transmissive to the (optical) signal and zeros outside of it, where we have no signal. Since the two extreme cases here are "all signal" vs. "no signal", we can represent that with zeros and ones. If we have an aperture that is partially transmissive in some areas, it will contain values between 0 and 1 there. In practical terms, this means that our $A$ will always have values between 0 and 1.

In [ ]:
# Define a function for the rectangular aperture.
# The expression below returns values of True or
# False, so we have to convert that into floats.
def rect2d(size):
    """Rectangluar aperture. size is a tupel (x,y)."""
    rect = (np.abs(xx) <= (size[0]/2)) * (np.abs(yy) <= (size[1]/2))
    return rect.astype('float')

In [ ]:
# Create a rectangular aperture
A = 1
T = (0.4, 0.2)   # x and y size of the rectangular aperture
rect_ap = rect2d(T)

# Display the full function
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xx, yy, rect_ap)
ax.set_title('$f(x, y)$ - rectangular aperture')
ax.set_xlabel('x')
ax.set_ylabel('y')

Oftentimes it will not give us any extra information to display a 2D function like this, but instead the amplitude of the function at each point will be indicated by different colors in an instance of `plt.imshow()`.

In [ ]:
# Display the rectangular aperture wtih imshow()
plt.figure(figsize=(5, 5))
plt.imshow(rect_ap)
plt.colorbar()

We can see how the the transmissive part of this aperture is filled with ones, while the opaque parts around it are filled with zeros.

### FT of a rectangular aperture

I will show the functions in the following both in a 3D surface plot as well as on a 2D image, just to give a feeling for what's going on. Also, remember that `plt.imshow()` cannot plot complex numbers, and as opposed to `plt.plot()` it will *not* default to plotting the real part, but will raise an error instead. We have to speficy what we want it to show us.

In [ ]:
# Calculate the FT
rec_ft = ft2d(padcplx(rect_ap))

# Plot
fig = plt.figure(figsize=(15,15))
plt.suptitle('$F(s,r)$')

ax1 = fig.add_subplot(221, projection='3d')
ax1.plot_surface(ss, rr, np.real(rec_ft), cmap=cm.coolwarm)
ax1.set_title('Real')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.view_init(10, -60)
ax1.set_zlim3d(-5000,5000)

ax2 = fig.add_subplot(222, projection='3d')
ax2.plot_surface(ss, rr, np.imag(rec_ft), cmap=cm.coolwarm)
ax2.set_title('Imaginary')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.view_init(30, -60)

ax3 = fig.add_subplot(223)
im1 = ax3.imshow(np.real(rec_ft))
ax3.set_title('Real')
fig.colorbar(im1)

ax4 = fig.add_subplot(224)
im2 = ax4.imshow(np.imag(rec_ft))
ax4.set_title('Imaginary')
fig.colorbar(im1)

This might not look like much at first, but everything is ok here. All we have to do is zoom in a little in order to see the parts that are interesting to us (see also my note in the first part of the notebook where I create the grids).  

Also, we can see that the imaginary part of the FT is just noise and not interesting to us right now.

In [ ]:
# Zoom into the FT of the rectangular aperture
zoomfac = 20     # half-size of the zoom box will be 1/zoomfac of total image
box = int(npix_pad/zoomfac)

# This is a smaller data array wiht our region of interest.
rec_ft_zoom = zoom(rec_ft, int(npix_pad/2), int(npix_pad/2), box)

# I also have to adjust our s and r grids to match the zoomed data size
sz = zoom(ss, int(npix_pad/2), int(npix_pad/2), box)
rz = zoom(rr, int(npix_pad/2), int(npix_pad/2), box)

# Plot
fig = plt.figure(figsize=(20,7))
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(sz, rz, np.real(rec_ft_zoom), cmap=cm.coolwarm)
ax1.set_title('$F(s,r)$')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.view_init(10, -60)
ax1.set_zlim3d(-5000,20000)

ax3 = fig.add_subplot(122)
im1 = ax3.imshow(np.real(rec_ft_zoom))
ax3.set_title('$F(s,r)$')
fig.colorbar(im1)

Nice! We can see very well in the right hand plot what the FT of a rectangular aperture looks like. Let's move on to different 2D functions $f(x,y)$ that we want to transform.

## Circular aperture

The circular aperture is a classic one, since the standard case for a telescope is a circular entrance pupil.

In [ ]:
# Define a function for a circular aperture
def circle_mask(im, xc, yc, rcirc):
    """Create a circular aperture centered on (xc, yc) with radius rcirc."""
    x, y = np.shape(im)
    newy, newx = np.mgrid[:y,:x]
    circ = (newx-xc)**2 + (newy-yc)**2 < rcirc**2
    return circ

In [ ]:
# Create a rectangular aperture
A = 1
rad = 0.7 * npix/2   # x and y size of the rectangular aperture
circ_ap = circle_mask(xx, int(npix/2), int(npix/2), rad)

# Plot
fig = plt.figure(figsize=(15,7))
plt.suptitle('$f(x, y)$ - circular aperture')

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(xx, yy, circ_ap)
ax1.set_xlabel('x')
ax1.set_ylabel('y')

ax2 = fig.add_subplot(122)
im1 = ax2.imshow(circ_ap)
#fig.colorbar(im1)

### FT of a circular aperture

In [ ]:
# Calculate the FT
circ_ft = ft2d(padcplx(circ_ap))

In [ ]:
# Plot
zoomfac = 30     # half-size of the zoom box will be 1/zoomfac of total image
box = int(npix_pad/zoomfac)

# This is a smaller data array wiht our region of interest.
circ_ft_zoom = zoom(circ_ft, int(npix_pad/2), int(npix_pad/2), box)

# I also have to adjust our s and r grids to match the zoomed data size
sz = zoom(ss, int(npix_pad/2), int(npix_pad/2), box)
rz = zoom(rr, int(npix_pad/2), int(npix_pad/2), box)

# Plot
fig = plt.figure(figsize=(20,7))
ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(sz, rz, np.real(circ_ft_zoom), cmap=cm.coolwarm)
ax1.set_title('$F(s,r)$')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.view_init(10, -60)

ax3 = fig.add_subplot(122)
im1 = ax3.imshow(np.real(circ_ft_zoom))
ax3.set_title('$F(s,r)$')
fig.colorbar(im1)

### Inverse FT

Let's see whether doing the inverse Fourier transform of the Fourier transform yields what we'd expect, like we did in the 1D case.

In [ ]:
# Take the inverse FT
circ_ft_back = ift2d(circ_ft)

In [ ]:
# Plot
circ_ft_back_zoom = zoom(circ_ft_back, int(npix_pad/2), int(npix_pad/2), int(npix/2))


fig = plt.figure(figsize=(15,7))
plt.suptitle('$f(x, y)$ - circular aperture')

ax1 = fig.add_subplot(121, projection='3d')
ax1.plot_surface(xx, yy, np.real(circ_ft_back_zoom))
ax1.set_xlabel('x')
ax1.set_ylabel('y')

ax2 = fig.add_subplot(122)
ax2.imshow(np.real(circ_ft_back_zoom))

Looking pretty good, on to the next one.

## Gaussian

## 2D sines and cosines

## Slits - two rectangles

## Ring

## Two dots

In [ ]:
#TODO: random apertures - hm, maybe better to put this in nb6, since that's where we get optics context:
# -> circular with struts and central obscuration, hexagonal, etc, etc, etc, ...

#TODO: all the 3D plots kinda look like shit, I need to improve them

In [ ]:
#TODO: want to look through this:
#http://www.robots.ox.ac.uk/~az/lectures/ia/lect2.pdf